In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("Example").getOrCreate()

In [3]:
df = spark.read.csv("College.csv",header = True,inferSchema=True)

In [7]:
df.head(1)

[Row(School='Abilene Christian University', Private='Yes', Apps=1660, Accept=1232, Enroll=721, Top10perc=23, Top25perc=52, F_Undergrad=2885, P_Undergrad=537, Outstate=7440, Room_Board=3300, Books=450, Personal=2200, PhD=70, Terminal=78, S_F_Ratio=18.1, perc_alumni=12, Expend=7041, Grad_Rate=60)]

In [6]:
df.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [8]:
from pyspark.ml.feature import VectorAssembler

In [9]:
df.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [10]:
assembler = VectorAssembler(inputCols=[ 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate'],outputCol="features")

In [16]:
output = assembler.transform(df)

In [13]:
from pyspark.ml.feature import StringIndexer

In [15]:
indexer = StringIndexer(inputCol = "Private",outputCol = "PrivateIndex")

In [17]:
output_fixed = indexer.fit(output).transform(output)

In [19]:
output_fixed.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- PrivateIndex: double (nullable = false)



In [20]:
final_data = output_fixed.select("features","PrivateIndex")

In [22]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [23]:
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier, GBTClassifier

In [24]:
from pyspark.ml import Pipeline

In [38]:
dtc = DecisionTreeClassifier(labelCol="PrivateIndex",featuresCol = "features")
rfc = RandomForestClassifier(numTrees=150,labelCol="PrivateIndex",featuresCol = "features")
gbt = GBTClassifier(labelCol="PrivateIndex",featuresCol = "features")

In [39]:
dtc_model=dtc.fit(train_data)
rfc_model=rfc.fit(train_data)
gbt_model=gbt.fit(train_data)

In [40]:
dtc_preds = dtc_model.transform(test_data)
rfc_preds = rfc_model.transform(test_data)
gbt_preds = gbt_model.transform(test_data)

In [41]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [42]:
binary_eval = BinaryClassificationEvaluator(labelCol= "PrivateIndex")

In [43]:
binary_eval.evaluate(dtc_preds)

0.9093370681605976

In [44]:
binary_eval.evaluate(rfc_preds)#increasing numTrees=150 insten of 20 make the score 0.97 to 0.98

0.9851540616246498

In [36]:
##gbt doesnt have default raw prediction column

In [34]:
binary_eval2 = BinaryClassificationEvaluator(labelCol= "PrivateIndex",rawPredictionCol="prediction")

In [35]:
binary_eval2.evaluate(gbt_preds)

0.9288515406162464

In [45]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [48]:
acc_eval = MulticlassClassificationEvaluator(labelCol="PrivateIndex",metricName="accuracy")

In [49]:
acc_eval.evaluate(rfc_preds)

0.944206008583691